In [1]:
!pip install psycopg2-binary requests python-doten

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.7 MB 1.7 MB/s eta 0:00:02
   --------------------------- ------------ 1.8/2.7 MB 2.4 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.7 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 2.5 MB/s eta 0:00:00


In [2]:
import psycopg2 as p
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [35]:
def get_db_connection():
    return p.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )

In [36]:
def get_new_requests():
    conn = get_db_connection()
    cur = conn.cursor()

    cur.execute("""
        SELECT id, name, phone, message, created_at
        FROM phonerequest
        WHERE jira_ticket_id IS NULL;
    """)

    rows = cur.fetchall()

    cur.close()
    conn.close()
    return rows


In [37]:
import requests

def create_jira_ticket(name, phone, message):
    url = os.getenv("JIRA_URL") + "/rest/api/3/issue"
    
    auth = (os.getenv("JIRA_EMAIL"), os.getenv("JIRA_API_TOKEN"))

    body = {
        "fields": {
            "project": {"key": "SUP"},   # your Jira project key
            "summary": f"Support Request from {name}",
            "description": f"Phone: {phone}\n\nMessage:\n{message}",
            "issuetype": {"name": "Task"}
        }
    }

    response = requests.post(url, json=body, auth=auth)
    response.raise_for_status()

    return response.json()["key"]   # Jira ticket ID


In [38]:
def update_request_with_ticket(request_id, ticket_id):
    conn = get_db_connection()
    cur = conn.cursor()

    cur.execute("""
        UPDATE phonerequest
        SET jira_ticket_id = %s
        WHERE id = %s
    """, (ticket_id, request_id))

    conn.commit()
    cur.close()
    conn.close()


In [39]:
def process_requests():
    requests = get_new_requests()

    for req in requests:
        req_id, name, phone, message, created_at = req

        print(f"Creating ticket for request #{req_id}...")

        ticket_id = create_jira_ticket(name, phone, message)

        update_request_with_ticket(req_id, ticket_id)

        print(f"✔ Created Jira ticket {ticket_id} for request {req_id}")


In [40]:
import time

while True:
    process_requests()
    time.sleep(900)   # 900 seconds = 15 minutes

OperationalError: connection to server at "aws-1-eu-west-2.pooler.supabase.com" (13.41.127.111), port 6543 failed: fe_sendauth: no password supplied
